In [10]:
import leafmap
from samgeo import SamGeo

In [11]:
# Configure local SkySat downloads directory and utilities
import os, glob
from IPython.display import display

parent_directory = '/Users/slxg3/Downloads/PhD/RA/ClimateTrace/skysat_downloads'
assert os.path.isdir(parent_directory), f'Directory not found: {parent_directory}'

In [12]:
# Discover city folders and .tif images
def list_city_image_paths(parent_dir):
    image_entries = []  # list of (city, image_path)
    for city in sorted([d for d in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, d))]):
        tif_paths = sorted(glob.glob(os.path.join(parent_dir, city, '*.tif')))
        for p in tif_paths:
            image_entries.append((city, p))
    return image_entries

all_city_folders = sorted([d for d in os.listdir(parent_directory) if os.path.isdir(os.path.join(parent_directory, d))])
images = list_city_image_paths(parent_directory)
print(f'Found {len(images)} images across {len(all_city_folders)} cities.')

Found 171 images across 33 cities.


In [13]:
sam = SamGeo(
    model_type="vit_h",
    automatic=True,
    sam_kwargs=None,
)

In [14]:
# Limit browsing to the first 10 images
BROWSE_IMAGES = images[:10]
print(f"Browsing {len(BROWSE_IMAGES)} images (first 10 or fewer if not available)")

Browsing 10 images (first 10 or fewer if not available)


In [15]:
# Helper to display a single image by index using SamGeo and the BROWSE_IMAGES subset
# Assumes `images` and `BROWSE_IMAGES` were created earlier

from IPython.display import display

def show_image(idx):
    idx = int(idx)
    if not BROWSE_IMAGES:
        print("No images found in BROWSE_IMAGES.")
        return
    if idx < 0 or idx >= len(BROWSE_IMAGES):
        print(f"Index {idx} out of range [0, {len(BROWSE_IMAGES)-1}].")
        return
    city, tif_path = BROWSE_IMAGES[idx]
    print(f"{idx+1}/{len(BROWSE_IMAGES)} | {city} | {os.path.basename(tif_path)}")

    # Use the global SamGeo instance created earlier
    try:
        sam
    except NameError:
        from samgeo import SamGeo
        globals()['sam'] = SamGeo(model_type="vit_h", automatic=True, sam_kwargs=None)

    sam.set_image(tif_path)
    m = sam.show_map()
    display(m)

In [16]:
# Add Next/Prev buttons to step through the first 10 images (BROWSE_IMAGES)
from ipywidgets import Button, HBox, VBox, Output

if len(BROWSE_IMAGES) == 0:
    print("No images found.")
else:
    out = Output()
    state = {'index': 0}  # use a small mutable state instead of nonlocal

    def render():
        with out:
            out.clear_output(wait=True)
            show_image(state['index'])

    def on_prev(_):
        if state['index'] > 0:
            state['index'] -= 1
            render()

    def on_next(_):
        if state['index'] < len(BROWSE_IMAGES) - 1:
            state['index'] += 1
            render()

    prev_btn = Button(description='Prev', button_style='')
    next_btn = Button(description='Next', button_style='primary')
    prev_btn.on_click(on_prev)
    next_btn.on_click(on_next)

    controls = HBox([prev_btn, next_btn])
    display(VBox([controls, out]))
    render()